In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [27]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 3

5


In [28]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=577, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)'],
 ['[\'R_draw\', \'B_win_by_Decision_Unanimous\', "R_Women\'s Feath

In [29]:
df = pd.read_csv("../data/ufc-master.csv")

In [30]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [31]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [32]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [33]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4232
4232
4232


In [34]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Karl Roberson,Marvin Vettori,210,-230,210.000000,43.478261,2020-06-13,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,0.00,0.66,0.0,2.0,1.0,2.0,0.0,0.0,1
201,Charles Rosa,Kevin Aguilar,170,-185,170.000000,54.054054,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,2.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
202,Andre Fili,Charles Jourdain,-220,200,45.454545,200.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.83,0.00,0.0,0.0,5.0,0.0,0.0,0.0,0
203,Jordan Espinosa,Mark De La Rosa,-167,157,59.880240,157.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,3.0,0.66,0.00,0.0,0.0,2.0,0.0,0.0,0.0,0
204,Mariya Agapova,Hannah Cifers,-335,305,29.850746,305.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,0.00,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.50,0.0,0.0,0.0,0.0,0.0,0.0,1
4428,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.00,0.0,1.0,0.0,1.0,0.0,1.0,0
4429,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
4430,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.00,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Brian Ortega,Chan Sung Jung,160,-200,160.000000,50.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Katlyn Chookagian,Jessica Andrade,130,-159,130.000000,62.893082,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Jimmy Crute,Modestas Bukauskas,-335,250,29.850746,250.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Claudio Silva,James Krause,120,-150,120.000000,66.666667,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Thomas Almeida,Jonathan Martinez,-139,110,71.942446,110.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Cortney Casey,Gillian Robertson,100,-120,100.000000,83.333333,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,4.0,0.0,0.75,0.0,1.0,0.0,5.0,0.0,0.0,1
196,Frank Camacho,Justin Jaynes,-340,280,29.411765,280.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1
197,Roxanne Modafferi,Lauren Murphy,-125,105,80.000000,105.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,0.0,0.66,0.0,0.0,0.0,1.0,1.0,0.0,1
198,Austin Hubbard,Max Rohskopf,160,-185,160.000000,54.054054,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,6.0,0.0,0.33,0.0,1.0,0.0,2.0,0.0,0.0,0


In [35]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [36]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Brian Ortega,Chan Sung Jung,160,-200,160.000000,50.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Katlyn Chookagian,Jessica Andrade,130,-159,130.000000,62.893082,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Jimmy Crute,Modestas Bukauskas,-335,250,29.850746,250.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Claudio Silva,James Krause,120,-150,120.000000,66.666667,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Thomas Almeida,Jonathan Martinez,-139,110,71.942446,110.000000,2020-10-17,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Cortney Casey,Gillian Robertson,100,-120,100.000000,83.333333,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,4.0,0.0,0.75,0.0,1.0,0.0,5.0,0.0,0.0,1
196,Frank Camacho,Justin Jaynes,-340,280,29.411765,280.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1
197,Roxanne Modafferi,Lauren Murphy,-125,105,80.000000,105.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,0.0,0.66,0.0,0.0,0.0,1.0,1.0,0.0,1
198,Austin Hubbard,Max Rohskopf,160,-185,160.000000,54.054054,2020-06-20,"Las Vegas, Nevada, USA",USA,Red,...,6.0,0.0,0.33,0.0,1.0,0.0,2.0,0.0,0.0,0


In [37]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [38]:
models[1][model_num]

'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)'

In [39]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [40]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [41]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [42]:
#1. set features
my_pos_features = ['R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [43]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_3
GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,
                           random_state=75)
['weight_class', 'B_Reach_cms', 'B_Welterweight_rank', 'B_total_title_bouts', 'R_avg_TD_pct', 'B_Bantamweight_rank', 'R_Featherweight_rank', 'no_of_rounds', 'B_draw', 'B_Stance', 'R_win_by_TKO_Doctor_Stoppage', 'R_current_win_streak', 'B_avg_SIG_STR_landed', 'country']
27


In [44]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [45]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [21]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    
    
    

Current best score is: 21357.719410233363
Feature: B_current_win_streak Score: 22582.96186240442
Feature: B_avg_TD_landed Score: 23039.03638513157
Feature: better_rank Score: 23716.12245896592
The best feature was better_rank.  It scored 23716.12245896592
Current best score is: 23716.12245896592
Feature: B_current_lose_streak Score: 23958.519889884017
Feature: B_current_win_streak Score: 24354.450111973583
Feature: B_longest_win_streak Score: 24746.67221622763
The best feature was B_longest_win_streak.  It scored 24746.67221622763
Current best score is: 24746.67221622763
Feature: B_win_by_Decision_Unanimous Score: 25039.469210820178
Feature: B_Middleweight_rank Score: 25507.462757645022
The best feature was B_Middleweight_rank.  It scored 25507.462757645022
Current best score is: 25507.462757645022
Feature: B_win_by_Decision_Unanimous Score: 25608.666128772416
Feature: R_wins Score: 26028.73463353137
The best feature was R_wins.  It scored 26028.73463353137
Current best score is: 26028

(3279, 35)

(3279,)

(3279, 2)

(200, 35)

...

(200, 35)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,160,-200,6.597020e-01,0.340298,0
1,130,-159,7.515074e-01,0.248493,1
2,-335,250,9.951751e-01,0.004825,0
3,120,-150,9.668769e-01,0.033123,1
4,-139,110,7.568759e-01,0.243124,1
...,...,...,...,...,...
195,100,-120,6.084503e-01,0.391550,1
196,-340,280,8.098215e-01,0.190178,1
197,-125,105,8.110044e-01,0.188996,1
198,160,-185,5.920577e-01,0.407942,0



          Number of matches: 200
          Number of bets: 194
          Number of winning bets: 109
          Number of losing bets: 85
          Number of underdog bets: 76
          Number of underdog wins: 31
          Number of underdog losses: 45
          Number of Favorite bets: 112
          Number of favorite wins: 76
          Number of favorite losses: 36
          Number of even bets: 6
          Number of even wins: 2
          Number of even losses: 4
          Profit: -240.0624854496455
          Profit per bet: -1.237435492008482
          Profit per match: -1.2003124272482275
          
          
HI
HI
HI
HI
HI


In [46]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3191, 55)

(3191,)

(3191, 2)

(200, 55)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,160,-200,0.920076,0.079924,0
1,130,-159,0.904637,0.095363,1
2,-335,250,0.009111,0.990889,0
3,120,-150,0.996087,0.003913,1
4,-139,110,0.999331,0.000669,1
...,...,...,...,...,...
195,100,-120,0.999771,0.000229,1
196,-340,280,0.798142,0.201858,1
197,-125,105,0.790271,0.209729,1
198,160,-185,0.895808,0.104192,0



          Number of matches: 200
          Number of bets: 161
          Number of winning bets: 94
          Number of losing bets: 67
          Number of underdog bets: 73
          Number of underdog wins: 32
          Number of underdog losses: 41
          Number of Favorite bets: 82
          Number of favorite wins: 58
          Number of favorite losses: 24
          Number of even bets: 6
          Number of even wins: 4
          Number of even losses: 2
          Profit: 1768.768899815616
          Profit per bet: 10.986142234879601
          Profit per match: 8.84384449907808
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [47]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [48]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [49]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [50]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=96, max_leaf_nodes=577, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=197,\n                       n_jobs=-1, random_state=75)",
  'GradientBoostingClassifier(max_depth=95, min_samples_leaf=6, n_estimators=195,\n                           random_state=75)',
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()',
  'GaussianNB(var_smoothing=1e-12)',
  'GaussianNB(var_smoothing=1e-12)'],
 ['[\'R_draw\', \'B_win_by_Decision_Unanimous\', "R_Women\'s Feath